In [1]:
import copy
import pandas as pd
import dataclasses as dc

![Clasificación](../img/classification.png)

Girona   0 - 1 Tenerife  
Tenerife 2 - 1 Girona

Tenerife   0 - 1 Las Palmas  
Las Palmas 2 - 1 Tenerife

Tenerife 4 - 0 Oviedo  
Oviedo   0 - 0 Tenerife

Las Palmas 1 - 3 Girona  
Girona     0 - 0 Las Palmas

Girona 2 - 1 Oviedo  
Oviedo 0 - 0 Girona

Oviedo     1 - 1 Las Palmas  
Las Palmas 2 - 1 Oviedo

In [2]:
class Team:
    TENERIFE = 'TEN'
    GIRONA = 'GIR'
    LAS_PALMAS = 'LPA'
    OVIEDO = 'OVI'

class Result:
    WIN = 'W'
    TIE = 'T'
    LOSE = 'L'

@dc.dataclass
class Allocation:
    team: Team
    result: Result
    points: int

In [3]:
teams = (Team.TENERIFE, Team.GIRONA, Team.LAS_PALMAS, Team.OVIEDO)
results = (Result.WIN, Result.TIE, Result.LOSE)
ppg = {Result.WIN: 3, Result.TIE: 1, Result.LOSE: 0}

init_points = {Team.TENERIFE: 69, Team.GIRONA: 67, Team.LAS_PALMAS: 67, Team.OVIEDO: 65}

INITIAL_POS = 4

In [4]:
untie_frame = pd.DataFrame(
    {Team.TENERIFE:   (0, 6, 0, 4),
     Team.GIRONA:     (0, 0, 4, 4),
     Team.LAS_PALMAS: (6, 1, 0, 4),
     Team.OVIEDO:     (1, 1, 1, 0)},
    index=(Team.TENERIFE, Team.GIRONA, Team.LAS_PALMAS, Team.OVIEDO)).T

untie_frame

,TEN,GIR,LPA,OVI
TEN,0,6,0,4
GIR,0,0,4,4
LPA,6,1,0,4
OVI,1,1,1,0


In [5]:
def make_combinations(teams, results, ppg):
    def make_combinations_(k=0, combination=[]):
        if k == len(teams):
            yield copy.copy(combination)
        else:
            for result in results:
                team = teams[k]
                combination[k] = Allocation(team, result, init_points[team] + ppg[result])
                yield from make_combinations_(k+1, combination)
    return make_combinations_(combination = [0] * len(teams))

In [6]:
def sort_combination(combination, untie_frame):
    def untie(chunk):
        teams = [item.team for item in chunk]
        untie_values = untie_frame.loc[teams][teams].sum(axis=1)
        return sorted(chunk, key=lambda v: untie_values[v.team], reverse=True)
        
    def sort_combination_(k=0, combination=[]):
        if k == len(combination):
            return combination
        current_points = combination[k].points
        for m in range(k+1, len(combination)):
            if combination[m].points != current_points:
                break
        else:
            m = k+1
        current_chunk = combination[k:m]
        if len(current_chunk) > 1:
            combination[k:m] = untie(current_chunk)
        return sort_combination_(k + len(current_chunk), combination)

    combination = sorted(combination, key=lambda v: v.points, reverse=True)
    return sort_combination_(combination=combination)

In [7]:
combinations = [sort_combination(c, untie_frame) for c in make_combinations(teams, results, ppg)]

In [12]:
staged_combinations = []
for comb_id, combination in enumerate(combinations, start=1):
    for position, allocation in enumerate(combination):
        a = dc.asdict(allocation)
        a['comb_id'] = comb_id
        a['position'] = INITIAL_POS + position
        staged_combinations.append(a)

In [13]:
df = pd.DataFrame(staged_combinations)

In [14]:
df

,team,result,points,comb_id,position
0,TEN,W,72,1,4
1,GIR,W,70,1,5
2,LPA,W,70,1,6
3,OVI,W,68,1,7
4,TEN,W,72,2,4
...,...,...,...,...,...
319,OVI,T,66,80,7
320,TEN,L,69,81,4
321,GIR,L,67,81,5
322,LPA,L,67,81,6


In [15]:
df.to_csv('../data/data.csv', index=False)